In [4]:
import pandas as pd
import numpy as np
from haversine import haversine

# Charger les datasets
customers = pd.read_csv("../Datasets/olist_customers_dataset.csv")
geolocation = pd.read_csv("../Datasets/olist_geolocation_dataset.csv")
orders = pd.read_csv("../Datasets/olist_orders_dataset.csv")
order_items = pd.read_csv("../Datasets/olist_order_items_dataset.csv")
products = pd.read_csv("../Datasets/olist_products_dataset.csv")
sellers = pd.read_csv("../Datasets/olist_sellers_dataset.csv")
payments = pd.read_csv("../Datasets/olist_order_payments_dataset.csv")
category_translation = pd.read_csv("../Datasets/product_category_name_translation.csv")
reviews = pd.read_csv("../Datasets/olist_order_reviews_dataset.csv")

# Fusionner les données
orders_customers = orders.merge(customers, on="customer_id")
orders_items = orders_customers.merge(order_items, on="order_id")
orders_items_products = orders_items.merge(products, on="product_id", how="left")
orders_items_sellers = orders_items_products.merge(sellers, on="seller_id", how="left")
payments_orders = payments.merge(orders_items_sellers, on="order_id", how="left")
payments_orders = payments_orders.merge(reviews, on="order_id", how="left")

# Nettoyer la géolocalisation
geolocation = geolocation.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean'
}).reset_index()

# Fusionner les coordonnées géographiques
payments_orders = payments_orders.merge(
    geolocation, 
    left_on='customer_zip_code_prefix', 
    right_on='geolocation_zip_code_prefix', 
    how='left'
).rename(columns={
    'geolocation_lat': 'geolocation_lat_client',
    'geolocation_lng': 'geolocation_lng_client'
})

payments_orders = payments_orders.merge(
    geolocation, 
    left_on='seller_zip_code_prefix', 
    right_on='geolocation_zip_code_prefix', 
    how='left'
).rename(columns={
    'geolocation_lat': 'geolocation_lat_vendeur',
    'geolocation_lng': 'geolocation_lng_vendeur'
})

# Calculer la distance
def calcul_distance(row):
    try:
        client_coord = (row['geolocation_lat_client'], row['geolocation_lng_client'])
        seller_coord = (row['geolocation_lat_vendeur'], row['geolocation_lng_vendeur'])
        return haversine(client_coord, seller_coord)
    except:
        return np.nan

payments_orders["distance_km"] = payments_orders.apply(calcul_distance, axis=1)

# Gestion des dates
date_cols = ["order_purchase_timestamp", "order_delivered_customer_date", "order_estimated_delivery_date"]
payments_orders[date_cols] = payments_orders[date_cols].apply(pd.to_datetime)

# Calcul du retard de livraison
payments_orders["retard_livraison_jours"] = (
    payments_orders["order_delivered_customer_date"] - 
    payments_orders["order_estimated_delivery_date"]
).dt.days

# Calcul historique des commandes
historique_commandes = payments_orders.groupby("customer_unique_id").agg(
    nombre_commandes=("order_id", "count"),
    montant_moyen_depense=("payment_value", "mean"),
    derniere_commande=("order_purchase_timestamp", "max")
).reset_index()

date_reference = historique_commandes["derniere_commande"].max()
historique_commandes["temps_depuis_derniere_commande_jours"] = (
    date_reference - historique_commandes["derniere_commande"]
).dt.days

# Fusion finale
summary = payments_orders.merge(historique_commandes, on="customer_unique_id", how="left")

# Agrégation des métriques
summary = summary.groupby(["customer_unique_id", "customer_city", "customer_state", "order_id"]).agg(
    nombre_produits=("product_id", "count"),
    nombre_categories=("product_category_name", "nunique"),
    vendeurs=("seller_id", "nunique"),
    vendeur_ville=("seller_city", "first"),
    vendeur_etat=("seller_state", "first"),
    moyen_paiement=("payment_type", "first"),
    tranche_paiement=("payment_installments", "mean"),
    prix_total=("payment_value", "sum"),
    frais_transport=("freight_value", "sum"),
    date_commande=("order_purchase_timestamp", "first"),
    note_moyenne_avis=("review_score", "mean"),
    nombre_commandes=("nombre_commandes", "first"),
    montant_moyen_depense=("montant_moyen_depense", "first"),
    temps_depuis_derniere_commande_jours=("temps_depuis_derniere_commande_jours", "first"),
    distance_km=("distance_km", "first"),
    retard_livraison_jours=("retard_livraison_jours", "first"),
    latitude_client=("geolocation_lat_client", "first"),
    longitude_client=("geolocation_lng_client", "first"),
    latitude_vendeur=("geolocation_lat_vendeur", "first"),
    longitude_vendeur=("geolocation_lng_vendeur", "first")
).reset_index()

# Ajout des nouvelles colonnes
summary['frequence_achat'] = summary['nombre_commandes'] / (summary['temps_depuis_derniere_commande_jours'] + 1)
summary['cout_moyen_par_produit'] = summary['prix_total'] / summary['nombre_produits']
summary['diversite_des_achats'] = summary['nombre_categories'] / summary['nombre_produits']
summary['impact_des_frais_de_transport'] = summary['frais_transport'] / summary['prix_total']


# Sauvegarde finale
summary.to_csv("Datas.csv", index=False)
summary.head()

,customer_unique_id,customer_city,customer_state,order_id,nombre_produits,nombre_categories,vendeurs,vendeur_ville,vendeur_etat,moyen_paiement,...,distance_km,retard_livraison_jours,latitude_client,longitude_client,latitude_vendeur,longitude_vendeur,frequence_achat,cout_moyen_par_produit,diversite_des_achats,impact_des_frais_de_transport
0,0000366f3b9a7992bf8c76cfdf3221e2,cajamar,SP,e22acc9c116caa3f2b7121bbb380d08e,1,1,1,piracicaba,SP,credit_card,...,110.568788,-5.0,-23.340235,-46.830140,-22.708702,-47.664701,0.008621,141.90,1.0,0.084567
1,0000b849f77a49e4a4ce2b2a4ca5be3f,osasco,SP,3594e05a005ac4d06a72673270ef9ec9,1,1,1,sao paulo,SP,credit_card,...,22.168364,-5.0,-23.559115,-46.787626,-23.490759,-46.583369,0.008403,27.19,1.0,0.304892
2,0000f46a3911fa3c0805444483337064,sao jose,SC,b33ec3b699337181488304f362a6b734,1,1,1,campo limpo paulista,SP,credit_card,...,516.939550,-2.0,-27.542880,-48.633426,-23.211746,-46.762875,0.001845,86.22,1.0,0.199722
3,0000f6ccb0745a6a4b88665a16c9f078,belem,PA,41272756ecddd9a9ed0180413cc22fb6,1,1,1,sao paulo,SP,credit_card,...,2481.290615,-12.0,-1.312214,-48.483159,-23.543150,-46.490101,0.003067,43.62,1.0,0.404172
4,0004aac84e0df4da2b147fca70cf8255,sorocaba,SP,d957021f1127559cd947b62533f484f7,1,1,1,jacarei,SP,credit_card,...,154.508100,-8.0,-23.505548,-47.469705,-23.302318,-45.971888,0.003413,196.89,1.0,0.085784


In [5]:
summary.columns

Index(['customer_unique_id', 'customer_city', 'customer_state', 'order_id',
       'nombre_produits', 'nombre_categories', 'vendeurs', 'vendeur_ville',
       'vendeur_etat', 'moyen_paiement', 'tranche_paiement', 'prix_total',
       'frais_transport', 'date_commande', 'note_moyenne_avis',
       'nombre_commandes', 'montant_moyen_depense',
       'temps_depuis_derniere_commande_jours', 'distance_km',
       'retard_livraison_jours', 'latitude_client', 'longitude_client',
       'latitude_vendeur', 'longitude_vendeur', 'frequence_achat',
       'cout_moyen_par_produit', 'diversite_des_achats',
       'impact_des_frais_de_transport'],
      dtype='object')